In [11]:
from magentic import prompt, ConfigDict, OpenaiChatModel
from typing import Optional, List, Dict
from dataclasses import dataclass
from datetime import date, timedelta, time
from pydantic import BaseModel

In [12]:
class Event(BaseModel):
    model_config = ConfigDict(openai_strict=True)
    name: str
    description: str
    long_description: str
    date: date
    importance: int

In [14]:
@prompt(
    """ 
        List at least 10 events in history between {start} and {end} that were major world events with a big economic impact.
        For the long description, describe the event in detail, narrate what happened, and include the impact on the economy.
        """,
    model=OpenaiChatModel("gpt-4o-mini-2024-07-18"),
    max_retries=3
)
def get_events(start, end) -> List[Event]: ...


years: List[int] = [1960, 1970, 1980, 1990, 2000, 2010, 2020]


events: List[Event] = []

for year in years:
    year_events = get_events(year, year+10)
    
    for year_event in year_events:
        events.append(year_event)

events

[Event(name='The Kennedy Assassination', description='The assassination of President John F. Kennedy', long_description='On November 22, 1963, President John F. Kennedy was assassinated in Dallas, Texas. The event shocked the nation and the world, leading to a profound period of mourning. Following the assassination, there was a significant upheaval in the political landscape of the United States. Economically, the immediate aftermath saw fluctuations in the stock market due to uncertainty, but the subsequent Vietnam War escalations and social reforms initiated by the Johnson administration further shaped economic policies.', date=datetime.date(1963, 11, 22), importance=9),
 Event(name='Great Society Programs', description="Launch of President Johnson's Great Society initiatives", long_description='In the mid-1960s, President Lyndon B. Johnson unveiled his Great Society programs aimed at eliminating poverty and racial injustice. Key initiatives included Medicare, Medicaid, and various 

In [15]:
import pandas as pd

In [16]:
class EventWithEconomicData(Event):
    unemployment_rate: Optional[float] = None
    unemployment_rate_6m: Optional[float] = None
    unemployment_rate_12m: Optional[float] = None
    unemployment_rate_18m: Optional[float] = None
    unemployment_rate_24m: Optional[float] = None
    gdp: Optional[float] = None
    gdp_6m: Optional[float] = None
    gdp_12m: Optional[float] = None
    gdp_18m: Optional[float] = None
    gdp_24m: Optional[float] = None
    cpi: Optional[float] = None
    cpi_6m: Optional[float] = None
    cpi_12m: Optional[float] = None
    cpi_18m: Optional[float] = None
    cpi_24m: Optional[float] = None
    oil_price: Optional[float] = None
    oil_price_6m: Optional[float] = None
    oil_price_12m: Optional[float] = None
    oil_price_18m: Optional[float] = None
    oil_price_24m: Optional[float] = None


def add_economic_data(
    events: List[Event],
    unemployment_file: str,
    gdp_file: str,
    cpi_file: str,
    oil_price_file: str,
) -> List[EventWithEconomicData]:
    # Read CSV files
    unemployment_df = pd.read_csv(
        unemployment_file, parse_dates=["DATE"], index_col="DATE"
    )
    gdp_df = pd.read_csv(gdp_file, parse_dates=["DATE"], index_col="DATE")
    cpi_df = pd.read_csv(cpi_file, parse_dates=["DATE"], index_col="DATE")
    oil_price_df = pd.read_csv(oil_price_file, parse_dates=["DATE"], index_col="DATE")

    # Function to get economic data for a given date
    def get_data(date, df, column_name):
        timestamp = pd.Timestamp(date)
        return df[column_name].asof(timestamp)

    # Process events
    return [
        EventWithEconomicData(
            **event.model_dump(),
            unemployment_rate=get_data(event.date, unemployment_df, "UNRATE"),
            unemployment_rate_6m=get_data(
                event.date + timedelta(days=180), unemployment_df, "UNRATE"
            ),
            unemployment_rate_12m=get_data(
                event.date + timedelta(days=365), unemployment_df, "UNRATE"
            ),
            unemployment_rate_18m=get_data(
                event.date + timedelta(days=545), unemployment_df, "UNRATE"
            ),
            unemployment_rate_24m=get_data(
                event.date + timedelta(days=730), unemployment_df, "UNRATE"
            ),
            gdp=get_data(event.date, gdp_df, "GDPC1"),
            gdp_6m=get_data(event.date + timedelta(days=180), gdp_df, "GDPC1"),
            gdp_12m=get_data(event.date + timedelta(days=365), gdp_df, "GDPC1"),
            gdp_18m=get_data(event.date + timedelta(days=545), gdp_df, "GDPC1"),
            gdp_24m=get_data(event.date + timedelta(days=730), gdp_df, "GDPC1"),
            cpi=get_data(event.date, cpi_df, "CORESTICKM159SFRBATL"),
            cpi_6m=get_data(event.date + timedelta(days=180), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_12m=get_data(event.date + timedelta(days=365), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_18m=get_data(event.date + timedelta(days=545), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_24m=get_data(event.date + timedelta(days=730), cpi_df, "CORESTICKM159SFRBATL"),
            oil_price=get_data(event.date, oil_price_df, "WTISPLC"),
            oil_price_6m=get_data(
                event.date + timedelta(days=180), oil_price_df, "WTISPLC"
            ),
            oil_price_12m=get_data(
                event.date + timedelta(days=365), oil_price_df, "WTISPLC"
            ),
            oil_price_18m=get_data(
                event.date + timedelta(days=545), oil_price_df, "WTISPLC"
            ),
            oil_price_24m=get_data(
                event.date + timedelta(days=730), oil_price_df, "WTISPLC"
            )
        )
        for event in events
    ]

In [17]:
events_with_economic_data = add_economic_data(events=events, unemployment_file="UNRATE.csv", gdp_file="GDPC1.csv", cpi_file="CORESTICKM159SFRBATL.csv", oil_price_file="WTISPLC.csv")

In [18]:
import csv
from typing import List


def save_events_to_csv(events: List[EventWithEconomicData], filename: str):
    # Define the field names
    fieldnames = [
        "name",
        "description",
        "long_description",
        "date",
        "importance",
        "unemployment_rate",
        "unemployment_rate_6m",
        "unemployment_rate_12m",
        "unemployment_rate_18m",
        "unemployment_rate_24m",
        "gdp",
        "gdp_6m",
        "gdp_12m",
        "gdp_18m",
        "gdp_24m",
        "cpi",
        "cpi_6m",
        "cpi_12m",
        "cpi_18m",
        "cpi_24m",
        "oil_price",
        "oil_price_6m",
        "oil_price_12m",
        "oil_price_18m",
        "oil_price_24m",
    ]

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write each event as a row
        for event in events:
            writer.writerow(
                {
                    "name": event.name,
                    "description": event.description,
                    "long_description": event.long_description,
                    "date": event.date,
                    "importance": event.importance,
                    "unemployment_rate": event.unemployment_rate,
                    "unemployment_rate_6m": event.unemployment_rate_6m,
                    "unemployment_rate_12m": event.unemployment_rate_12m,
                    "unemployment_rate_18m": event.unemployment_rate_18m,
                    "unemployment_rate_24m": event.unemployment_rate_24m,
                    "gdp": event.gdp,
                    "gdp_6m": event.gdp_6m,
                    "gdp_12m": event.gdp_12m,
                    "gdp_18m": event.gdp_18m,
                    "gdp_24m": event.gdp_24m,
                    "cpi": event.cpi,
                    "cpi_6m": event.cpi_6m,
                    "cpi_12m": event.cpi_12m,
                    "cpi_18m": event.cpi_18m,
                    "cpi_24m": event.cpi_24m,
                    "oil_price": event.oil_price,
                    "oil_price_6m": event.oil_price_6m,
                    "oil_price_12m": event.oil_price_12m,
                    "oil_price_18m": event.oil_price_18m,
                    "oil_price_24m": event.oil_price_24m,
                }
            )


# Example usage:
# events = [EventWithEconomicData(...), EventWithEconomicData(...), ...]
# save_events_to_csv(events, 'economic_data.csv')

In [19]:
save_events_to_csv(events_with_economic_data, "events_with_economic_data_2.csv")